<a href="https://colab.research.google.com/github/koya-jp/AA-google-colab-kohya/blob/master/convert_diffusers_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Driveに接続
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#スクリプトを格納するフォルダを作成
!mkdir -p /content/script

#Driveから変換スクリプトをコピー
!cp /content/drive/MyDrive/StableDiffusion/script/convert_diffusers/convert_diffusers20_original_sd.py /content/script
!cp /content/drive/MyDrive/StableDiffusion/script/convert_diffusers/model_util.py /content/script


In [ ]:
!pip install -U diffusers[torch]==0.10.2

%cd /content/script

#モデルデータを変換する
!python convert_diffusers20_original_sd.py /content/drive/MyDrive/StableDiffusion/Model/chilledremix/chilled_remix_v2.safetensors /content/drive/MyDrive/StableDiffusion/Model/chilledremix --v1 --reference_model runwayml/stable-diffusion-v1-5

In [ ]:
#@title Modelが生成されたかどうかをテスト { display-mode: "both" }

from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from diffusers.models import AutoencoderKL
import torch

#画像生成に使うモデルデータ
model_id = "/content/drive/MyDrive/StableDiffusion/Model/chilledremix"
#画像生成に使うVAE
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse")
#画像生成に使うスケジューラー
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, vae=vae, custom_pipeline="lpw_stable_diffusion")

#LoRAを読み込む
pipe = load_safetensors_lora(pipe, '/content/drive/MyDrive/StableDiffusion/Lora/flat2.safetensors',alpha=-1.0)

pipe = pipe.to("cuda")